[Here is Wikipedia's list of all HTTP response codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
+ 100's: information
+ 200's: success           
+ 300's: redirects
+ 400's + 500's: errors

Perhaps familiar:  404 <br>
Especially fun:  418


In [5]:
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mp
import matplotlib.animation as animation
from matplotlib import style
import datetime as dt
# style.use('seaborn-v0_8-dark-palette')

import requests;


In [13]:
# %matplotlib qt
# NECESSARY SO JUPYTER SHOWS LIVE PLOT!!!

In [39]:

def ctlPower(url, switch):
    page = "get"
    param = "ctl-power"
    if switch == "on" or switch == 1:
        message = "=on"
    elif switch == "off" or switch == 0:
        message = "=off"
    else:
        print("Invalid input: ")
        print(switch)
        return False
    query = url+page+"?"+param+message
    result = requests.get(query)
    contents = result.text
    if "SUCCESS" in contents:
        return True
    else:
        return False
    

def ctlBlinks(url, blinks):
    page = "get"
    param = "ctl-blinks"
    message = "=" + str(blinks)
    query = url+page+"?"+param+message
    result = requests.get(query)
    contents = result.text
    if "SUCCESS" in contents:
        return True
    else:
        return False


def jsonDataRead(url):
    page = "get"
    param = "read-data"
    message = ""
    query = url+page+"?"+param+message
    result = requests.get(query)
    # contents = result.text
    json_contents = result.json()
    timestamp = dt.datetime.now().strftime('%H:%M:%S.%f')
    time_data = [timestamp, json_contents]
    # print(json_contents)
    return time_data


In [21]:
jsonDataRead("http://172.28.118.30/")

['19:13:34.572332', {'time': 421741, 'button status': 0, 'thermistor': 2000}]

In [34]:
# This function is called periodically from FuncAnimation
def animate(i, xs, ys, xs1, ys1, squish_bool, url):
    

    # Read button status from ESP32
    timestamp, data = jsonDataRead(url)
    # print(buttonStatus)

    # Add read time and button status to lists
    xs.append(timestamp)
    ys.append(data["button status"])

    xs1.append(timestamp)
    ys1.append(data['thermistor'])

     
    # plt.locator_params(axis='x', nbins=6)

    if squish_bool:
        pass
    else:
        # Limit x and y lists to 20 items, optional
        xs = xs[-20:]
        ys = ys[-20:]

        xs1 = xs1[-20:]
        ys1 = ys1[-20:]

    # Draw x and y lists
    ax.clear()
    ax.plot(xs, ys)

    ax1.clear()
    ax1.plot(xs1, ys1)
    

    # xticks = ax.get_xticks()
    # ax.set_xticks(xticks[::len(xticks) // 4]) # set new tick positions
    # ax.tick_params(axis='x', rotation=30) # set tick rotation
    # ax.margins(x=0) # set tight margins

    ax.xaxis.set_major_locator(plt.MaxNLocator(5))
    ax1.xaxis.set_major_locator(plt.MaxNLocator(5))

    # Format plot
    plt.xticks(rotation=45, ha='right')
    plt.subplots_adjust(bottom=0.30)
    plt.title('Thermistor Reading')
    plt.ylabel('Temperature (deg C)')



In [ ]:
ctlPower(url)


In [46]:
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time
# style.use('seaborn-v0_8-dark-palette')


def startplot():
    # Create figure for plotting
    fig = plt.figure()
    ax = fig.add_subplot(2, 1, 1)
    ax1 = fig.add_subplot(2, 1, 2)
    xs = []
    ys = []
    xs1 = []
    ys1 = []

    url = "http://172.28.118.30/"

    ctlPower(url, "off")

    time.sleep(5)

    ctlPower(url, "on")

    time.sleep(5)

    ctlBlinks(url, 3)


    # Initialize communication with TMP102

    # Set up plot to call animate() function periodically

    ani = animation.FuncAnimation(fig, animate, fargs=(xs, ys, xs1, ys1, False, url), interval=200)
    plt.show()

In [47]:
startplot()

/var/folders/t2/8szqbhws4zsc41hzthbc_6bw0000gn/T/ipykernel_90998/2852146132.py:35: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = animation.FuncAnimation(fig, animate, fargs=(xs, ys, xs1, ys1, False, url), interval=200)
/Users/allisonmarten/anaconda3/lib/python3.10/site-packages/matplotlib/animation.py:880: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you output the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(
